In [9]:
# import all necessary libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from math import log
import networkx as nx
from igraph import Graph, plot
import igraph as ig
from math import log
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import torch
import torch
import torch.nn.functional as F
from torch_geometric.nn import GAE, GCNConv
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from torch_geometric.utils import from_networkx


In [2]:
%pip install torch torch-geometric torch-scatter torch-sparse

Note: you may need to restart the kernel to use updated packages.


Instead of using handcrafted features (like degree, clustering coefficient, etc.), we use Graph Autoencoders to learn node embeddings directly from the graph structure + features.

In [3]:
file_path = 'featured_dataset.csv'
df = pd.read_csv(file_path)


In [4]:
df.shape

(3532, 17)

In [5]:
df.head(10)

,Node,Degree,ClusteringCoefficient,BetweennessCentrality,ClosenessCentrality,PageRank,EigenvectorCentrality,Eccentricity,Constraint,MeanNeighborDegree,LocalClusterSizes,LocalClusterDensities,sorensen_indices,participation_coefficient,CommunitySize,CommunityDensity,CommunityAssortativity
0,1,0.700547,0.419118,-0.057036,0.261376,1.000118,0.000006,1.043499,1.048030,-1.206914,-1.187895,0.047468,0.128317,1.078821,-0.849481,0.042446,-0.133841
1,2,0.597675,0.888889,-0.057268,0.261258,1.000100,0.000002,1.043499,1.100383,-1.206914,-1.187895,0.047468,0.122203,1.073933,-0.849481,0.042446,-0.133841
2,3,0.700547,0.632353,-0.057130,0.261376,1.000113,0.000007,1.043499,1.048897,-1.206914,-1.187895,0.047468,0.143140,1.078447,-0.849481,0.042446,-0.133841
3,4,0.597675,0.866667,-0.057259,0.261258,1.000116,0.000002,1.043499,1.149038,-1.206914,-1.187895,0.047468,0.110459,1.074068,-0.849481,0.042446,-0.133841
4,5,0.643779,0.333333,-0.057085,0.261308,1.000101,0.000012,1.043499,1.056431,-1.206914,-1.187895,0.047468,0.119667,1.076862,-0.849481,0.042446,-0.133841
5,6,0.530000,0.933333,-0.057273,0.261190,1.000088,0.000002,1.043499,1.189174,-1.206914,-1.187895,0.047468,0.126051,1.059941,-0.849481,0.042446,-0.133841
6,7,0.740272,0.431579,-0.042650,0.274769,1.000132,0.000269,1.043499,1.050490,-0.829093,-0.830723,0.054209,0.090114,1.079451,-0.849481,0.042446,-0.133841
7,8,0.564852,0.678571,-0.057251,0.261224,1.000115,0.000002,1.043499,1.170652,-1.206914,-1.187895,0.047468,0.113541,1.069639,-0.849481,0.042446,-0.133841
8,9,1.119786,0.397243,-0.055862,0.262055,1.000302,0.000023,1.043499,1.023864,-1.206914,-1.187895,0.047468,0.156285,1.081540,-0.849481,0.042446,-0.133841
9,10,0.597675,0.822222,-0.057270,0.261258,1.000068,0.000008,1.043499,1.062901,-1.206914,-1.187895,0.047468,0.148043,1.072578,-0.849481,0.042446,-0.133841


In [ ]:
# df_dropped is your feature dataframe
df_features = df.sort_values(by='Node')
x = torch.tensor(df_features.drop(columns=['Node']).values, dtype=torch.float)
# Extract node features and ensure order by node index and edges
G = nx.read_edgelist("facebook_combined.txt", nodetype=int)

In [ ]:
# Convert NetworkX graph to PyG Data format
pyg_graph = from_networkx(nx.Graph(G))

In [13]:
pyg_graph.x = x

In [ ]:
# Define Graph Autoencoder model
from torch_geometric.nn import GCNConv

class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        return self.conv2(x, edge_index)

In [ ]:
# Train the GAE model
from torch_geometric.nn import GAE

# Initialize model
out_channels = 32  # embedding size
model = GAE(Encoder(pyg_graph.num_node_features, out_channels))

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 200

model.train()
for epoch in range(epochs):
    optimizer.zero_grad()
    z = model.encode(pyg_graph.x, pyg_graph.edge_index)
    loss = model.recon_loss(z, pyg_graph.edge_index)
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')

RuntimeError: index 3532 is out of bounds for dimension 0 with size 3532